It seems the current [high scoring script][1] is written in R using H2O. So let us do one in python using XGBoost. 

Thanks to [this script][2] for feature engineering ideas. 

We shall start with importing the necessary modules


  [1]: https://www.kaggle.com/gospursgo/two-sigma-connect-rental-listing-inquiries/h2o-starter-pack/run/835757
  [2]: https://www.kaggle.com/aikinogard/two-sigma-connect-rental-listing-inquiries/random-forest-starter-with-numerical-features

In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
# import xgboost as xgb
# from sklearn import model_selection, preprocessing, ensemble
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from XGBoostPackage import xgbClass
from CrossValidation2 import CVScore
from Stacking import Ensemble, EnsembleClassifier
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
X_train=pd.read_csv('X_train2.csv').iloc[:,1:]
#X_test=pd.read_csv('X_test2.csv')
y_train=pd.read_csv('y_train.csv')['interest_level'].ravel()

In [8]:
n_folds=5
base_models=[[RandomForestClassifier(n_estimators=900, max_features='sqrt') for i in np.arange(n_folds)],\
            [xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321) \
             for i in np.arange(n_folds)], 
            [ExtraTreesClassifier(n_estimators=1000, max_features='sqrt') for i in np.arange(n_folds)]]
ensembleObj=EnsembleClassifier(n_folds=n_folds, n_class=3, stacker=xgbClass(objective='multi:softprob', \
                                                                         eva_metric="mlogloss", \
                                           colsample_bytree=.7, eta=.1, max_depth=6, min_child_weight=1, \
                                           num_class=3,subsample=.7, silent=1), \
                     base_models=base_models, is_TimeSeries=False, random_state=17)#,\
ensembleObj.fit(X_train=train_X, y_train=train_y)

startig training model 0, training set 0
startig training model 0, training set 1
startig training model 0, training set 2
startig training model 0, training set 3
startig training model 0, training set 4
startig training model 1, training set 0
startig training model 1, training set 1
startig training model 1, training set 2
startig training model 1, training set 3
startig training model 1, training set 4
startig training model 2, training set 0
startig training model 2, training set 1
startig training model 2, training set 2
startig training model 2, training set 3
startig training model 2, training set 4


In [9]:
pred=ensembleObj.predict_proba(X_test=test_X)

startig predicting model 0, training set 0
startig predicting model 0, training set 1
startig predicting model 0, training set 2
startig predicting model 0, training set 3
startig predicting model 0, training set 4
startig predicting model 1, training set 0
startig predicting model 1, training set 1
startig predicting model 1, training set 2
startig predicting model 1, training set 3
startig predicting model 1, training set 4
startig predicting model 2, training set 0
startig predicting model 2, training set 1
startig predicting model 2, training set 2
startig predicting model 2, training set 3
startig predicting model 2, training set 4


In [11]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high_ExT", "medium_ExT", "low_ExT"]
out_df["listing_id"] = X_test.listing_id.values
out_df.to_csv("apr16_1_Stack.csv", index=False)

startig predicting model 0, training set 0
startig predicting model 0, training set 1
startig predicting model 0, training set 2
startig predicting model 0, training set 3
startig predicting model 0, training set 4
startig predicting model 1, training set 0
startig predicting model 1, training set 1
startig predicting model 1, training set 2
startig predicting model 1, training set 3
startig predicting model 1, training set 4
startig predicting model 2, training set 0
startig predicting model 2, training set 1
startig predicting model 2, training set 2
startig predicting model 2, training set 3
startig predicting model 2, training set 4
startig predicting model 0, training set 0
startig predicting model 0, training set 1
startig predicting model 0, training set 2
startig predicting model 0, training set 3
startig predicting model 0, training set 4
startig predicting model 1, training set 0
startig predicting model 1, training set 1
startig predicting model 1, training set 2
startig pre

KeyboardInterrupt: 